In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from consts import *

tqdm.pandas()


F_IN_BIO = get_filename(TT.BIO, EMBED_PROVIDER.TURKISH_BERT_TWEET)
F_IN_CAP = get_filename(TT.CAP, EMBED_PROVIDER.TURKISH_BERT_TWEET, MEAN_METHOD.MEAN)
F_OUT = get_filename(TT.ALL, EMBED_PROVIDER.TURKISH_BERT_TWEET, MEAN_METHOD.MEAN)

print(F_IN_BIO)
print(F_IN_CAP)
print(F_OUT)

BIO_RATIO = 0.1

df_ub = pd.read_parquet(F_IN_BIO)
df_uc = pd.read_parquet(F_IN_CAP)

data/embeddings/turkishbertweet/ub_em.parquet
data/embeddings/turkishbertweet/uc_em_mean.parquet
data/embeddings/turkishbertweet/ue_em_mean.parquet


In [2]:
df_uc

,username,embeds
0,deparmedya,"[-0.127641715328483, 0.024034431991715205, 0.0..."
1,kafesfirin,"[-0.15189578607678414, 0.01873495416449649, 0...."
2,vimerang,"[-0.11766718434435981, 0.027429566555656492, 0..."
3,mustafa_yalcinn38,"[-0.11503724068403244, 0.02811199149582535, 0...."
4,drtubagunebak,"[-0.10723341490541186, 0.0396484497302611, 0.0..."
...,...,...
3465,woodtechistanbul,"[-0.06623451174902065, 0.05926215180328914, 0...."
3466,netsvadi,"[-0.10364578015037945, 0.027395657556397574, 0..."
3467,herbisiatolye,"[-0.08824355659946319, 0.03670846289324184, 0...."
3468,nestleturkiye,"[-0.13492713432226863, 0.03473074611143342, 0...."


In [3]:
df_ub

,username,biography,embeds
0,deparmedya,<hashtag> mediaplanning </hashtag> <hashtag> m...,"[-0.14732463657855988, 0.012161445803940296, -..."
1,kafesfirin,<emoji> yuvarlak_raptiye </emoji> söğütözü <em...,"[-0.18014419078826904, 0.0010251292260363698, ..."
2,vimerang,dijital i̇letişim yönetimi <emoji> klaket_taht...,"[-0.13595470786094666, 0.004898972809314728, -..."
3,mustafa_yalcinn38,talas belediye başkanı,"[-0.12611980736255646, -0.003150859847664833, ..."
4,drtubagunebak,<emoji> kadın_sağlık_çalışanı_açık_ten_tonu </...,"[-0.18248912692070007, -0.014781055971980095, ..."
...,...,...,...
3248,bs_forklift,kuruluş tarihi : 01.10.2014,"[-0.13339956104755402, 0.012177901342511177, -..."
3249,woodtechistanbul,avrasya'nın lider ağaç i̇şleme teknolojileri f...,"[-0.16302429139614105, 0.017178848385810852, 0..."
3250,netsvadi,<emoji> karla_kaplı_dağ </emoji> bornova kayad...,"[-0.16384869813919067, -0.010935492813587189, ..."
3251,herbisiatolye,<emoji> fide </emoji> şifalı bitkiler atölyesi...,"[-0.17341944575309753, -0.00872002448886633, -..."


In [4]:
df_combined = pd.merge(df_ub, df_uc, how='outer', on='username',suffixes=['_bio','_captions'])
df_combined = df_combined[['username','embeds_bio', 'embeds_captions']]
df_combined = df_combined.where(df_combined.notna(), None)


df_combined

,username,embeds_bio,embeds_captions
0,01burdaavm,"[-0.17908932268619537, 0.03460580110549927, 0....","[-0.13164054636250844, 0.030631494407840142, 0..."
1,1001sanat,"[-0.1366528570652008, -0.005800200160592794, -...","[-0.08822240077835672, 0.03321502571976881, 0...."
2,1924istanbul,"[-0.17066171765327454, 0.0033168462105095387, ...","[-0.12145432050932538, 0.034702123898155565, 0..."
3,1983beyoglu,"[-0.17637930810451508, -0.0007726665353402495,...","[-0.08486703825848443, 0.039586939429864285, 0..."
4,1dil1insan,"[-0.15181341767311096, 0.0014555572997778654, ...","[-0.15320842606680735, 0.019564709486439825, 4..."
...,...,...,...
3469,zpor,"[-0.16634802520275116, -0.010222718119621277, ...","[-0.14896014354058673, 0.005555204793095722, 0..."
3470,zuhaldentakilar,"[-0.19072918593883514, -0.003291164292022586, ...","[-0.1424271766628538, 0.014290992479904422, 0...."
3471,zulaoyun,"[-0.1636439859867096, 0.013511224649846554, 0....","[-0.1409842789794008, 0.013968864902077863, 0...."
3472,zulaoyunespor,"[-0.16551578044891357, 0.014706576243042946, 0...","[-0.10678398323555788, 0.03804382449015975, 0...."


In [5]:
import numpy as np


def embed_combinator(row):

    e_bio = row['embeds_bio']
    e_cap = row['embeds_captions']

    if e_bio is None and e_cap is None:
        return None


    if e_bio is None:
        return e_cap
    
    if e_cap is None:
        return e_bio

    return np.array(e_bio) * BIO_RATIO + np.array(e_cap) * (1-BIO_RATIO)



df_combined['embeds_wavg'] = df_combined.progress_apply(embed_combinator,axis=1)
df_combined = df_combined[['username','embeds_wavg']]
df_combined = df_combined.dropna()

df_combined['embeds_wavg'] = df_combined['embeds_wavg'].apply(lambda x: list(x))
df_combined

  0%|          | 0/3474 [00:00<?, ?it/s]

,username,embeds_wavg
0,01burdaavm,"[-0.13638542399487713, 0.031028925077606056, 0..."
1,1001sanat,"[-0.09306544640704113, 0.02931350313173265, 0...."
2,1924istanbul,"[-0.1263750602237203, 0.03156359612939096, 0.0..."
3,1983beyoglu,"[-0.0940182652430875, 0.035550978833343835, 0...."
4,1dil1insan,"[-0.1530689252274377, 0.01775379426777363, -0...."
...,...,...
3469,zpor,"[-0.15069893170680318, 0.003977412501824022, 0..."
3470,zuhaldentakilar,"[-0.14725737759045193, 0.012532776802711722, 0..."
3471,zulaoyun,"[-0.14325024968013167, 0.013923100876854732, 0..."
3472,zulaoyunespor,"[-0.11265716295689346, 0.035710099665448074, 0..."


In [6]:
df_combined.to_parquet(F_OUT)